# Data Cleaning

This iPython Notebook contains functions for cleaning up sales history export data files. This allows them to be used in the Inventory Suggestions program.

Listed below are the libraries used in this file. If it is ever run offline, these python libraries will need to be installed.

In [ ]:
import pandas as pd
import re

This file supports loading data from Google Drive. If you would like to use Google Drive for storing data, upload it and run the cell below. If not, don't run the cell. You can delete it, or stop it from running by putting # characters at the beginning of each line.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## File Paths

This code block tells the program where to look for the data file to clean. If you are using Google Drive, the `data_folder_path` variable should begin with "drive/MyDrive/", with any further subfolders listed after. For example:
```
# data folder path
data_folder = "drive/MyDrive/data/"
```
If you would rather simply upload the data files into Google Colab, you can put them in a folder:
```
# data folder path
data_folder = "data/"
```
Or, if you just put them in the default files directory, use the following:
```
# data folder path
data_folder = "./"
```
The period indicates we are using the current folder.

Then, input the name of the sales history file you would like to use into the variables below. For example:
```
# file path for sales history
sales_history_filename = data_folder + "Popcorn Sales History.csv"
```

In [ ]:
data_folder_path = "drive/MyDrive/data/"
file_name = "sales_history.csv"

## Dictionaries

These dictionaries replace misspelled or redundant words with synonyms. The word on the left side of the colon will be replaced with the word on the right. If you add new replacements to any of the dictionaries, be sure all pairs of replacements are separated by commas.

In [ ]:
# known synonyms for products & variants
PRODUCTS_SYNONYMS = {'redacted': 'redacted'}
VARIANTS_SYNONYMS = {'redacted': 'redacted'}
# other replacements for products & variants
PRODUCTS_REPLACEMENTS = {'redacted': 'redacted'}
VARIANTS_REPLACEMENTS = {'redacted': 'redacted'}

## Functions
The functions below are essential to the operation of the data cleaning program. They should only be modified by someone who definitely knows what they're doing.

In [ ]:
def adjust_dates(df, excel=False):
    """Convert the 'DAY Date', 'MONTH Date', and 'YEAR NUMBER Date' columns 
    into a single 'Date' columns containing a datetime64.

    Parameters:
    df (pandas.DataFrame): a dataframe constructed from popcorn data

    Returns:
    pandas.DataFrame: the same dataframe, but with the date columns converted 
        to one column
    """
    if excel:
        df['Date'] = pd.to_datetime({'year': df['YEAR NUMBER Date'],
                                 'month': pd.DatetimeIndex(df['MONTH Date']).month,
                                 'day': pd.DatetimeIndex(df['DAY Date']).day})
    else:
        df['Date'] = pd.to_datetime(df['DAY Date'], format="%m/%d/%Y")
    df.drop('YEAR NUMBER Date', axis=1, inplace=True)
    df.drop('MONTH Date', axis=1, inplace=True)
    df.drop('DAY Date', axis=1, inplace=True)
    return df

In [ ]:
def fix_product_variant_names(df):
    """Normalize the entries in the 'Product' and 'Variant' columns. This uses
    the synonyms and replacements given at the top of the file. 
    
    Parameters:
    df (pandas.DataFrame): a dataframe constructed from popcorn data

    Returns:
    pandas.DataFrame: the same dataframe, but with an attempted normalization 
        of the 'Product' and 'Variant' columns
    """
    # removing extra numbers
    df['Product name'] = [x.split(' / ')[0].lower() for x in list(df['Product name'])]
    df['Variant name'] = [x.split(' / ')[0].lower() for x in list(df['Variant name'])]
    # swapping old-format product/variant pairs -- assumes old format starts with "popcorn"
    for i, row in enumerate(df.iterrows()):
        if row[1]['Product name'].split(' ')[0] == 'popcorn' and row[1]['Product name'].split(' ')[1] != 'tower':
            temp = row[1]['Product name'][8:]
            df.loc[i, 'Product name'] = row[1]['Variant name']
            df.loc[i, 'Variant name'] = temp
    # removing any variants from the product name
    for i, row in enumerate(df.iterrows()):
        if row[1]['Variant name'] in row[1]['Product name']:
            df.loc[i, 'Product name'] = row[1]['Product name'].replace(row[1]['Variant name'], '').strip()
    # swapping synonyms
    for i, row in enumerate(df.iterrows()):
        for old, new in PRODUCTS_SYNONYMS.items():
            if old in row[1]['Product name'] and row[1]['Product name'] != new:
                df.loc[i, 'Product name'] = row[1]['Product name'].replace(old, new).strip()
        for old, new in VARIANTS_SYNONYMS.items():
            if old in row[1]['Variant name'] and row[1]['Variant name'] != new:
                df.loc[i, 'Variant name'] = row[1]['Variant name'].replace(old, new).strip()
    # swapping replacements
    for i, row in enumerate(df.iterrows()):
        for old, new in PRODUCTS_REPLACEMENTS.items():
            if old in row[1]['Product name'] and row[1]['Product name'] != new:
                df.loc[i, 'Product name'] = row[1]['Product name'].replace(old, new).strip()
        for old, new in VARIANTS_REPLACEMENTS.items():
            if old in row[1]['Variant name'] and row[1]['Variant name'] != new:
                df.loc[i, 'Variant name'] = row[1]['Variant name'].replace(old, new).strip()
    # remove popcorn from the end of product names
    for i, row in enumerate(df.iterrows()):
        if row[1]['Product name'].split(' ')[-1]=="popcorn":
            df.loc[i, 'Product name'] = row[1]['Product name'][:-8]
    return df

In [ ]:
def replace_missing_order_locations(df):
    """This function substitutes 0 for all the NaN location IDs.

    Parameters:
    df (pandas.DataFrame): a dataframe constructed from popcorn data

    Returns:
    pandas.DataFrame: the same dataframe, but with all NaN location IDs
        replaced with 0
    """
    df['Order Location Id'].fillna(0, inplace=True)
    return df

## Main Program Functionality

The cell below contains the main functionality of the program. Running this cell (as long as all other needed cells have already been run) will clean the sales history file at the given file location and name. 

This prints out all unique products and variants. You may want to check the results to be sure there are not synonyms that need to be added to the dictionaries at the top of the file.

In [ ]:
sales_history_file = data_folder_path+file_name

non_letters = re.compile('[^a-zA-Z0-9]')

name = non_letters.sub('', ''.join(sales_history_file.split('.')[:-1]))[7:]

if sales_history_file.split('.')[-1] == 'xlsx':
    sales_history = pd.read_excel(sales_history_file)
elif sales_history_file.split('.')[-1] == 'csv':
    sales_history = pd.read_csv(sales_history_file)
else:
    raise ValueError(f"File type not supported for file: {sales_history_file}")

sales_history = adjust_dates(sales_history)
sales_history = fix_product_variant_names(sales_history)
sales_history = replace_missing_order_locations(sales_history)

products_string = '\n'.join(tuple(sorted(list(set(sales_history['Product name'])))))
print(f"Products:\n{products_string}\n")
variants_string = '\n'.join(tuple(sorted(list(set(sales_history['Variant name'])))))
print(f"Variants:\n{variants_string}\n")

sales_history.to_csv(f"{data_folder_path}{name}.csv", index=False)